[Reference](https://levelup.gitconnected.com/say-goodbye-to-slow-analytic-queries-how-to-use-duckdb-in-python-for-lightning-fast-results-7c0bc2a42527)

In [1]:
pip install duckdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import duckdb

# create a new DuckDB database
con = duckdb.connect(database=':memory:')

# create a table
con.execute('CREATE TABLE users (id INTEGER, name VARCHAR(255))')

# insert some data
con.execute("INSERT INTO users VALUES (1, 'John')")
con.execute("INSERT INTO users VALUES (2, 'Jane')")

# query the data
result = con.execute("SELECT * FROM users").fetchall()
for row in result:
    print(row)

(1, 'John')
(2, 'Jane')


In [3]:
import duckdb

conn = duckdb.connect(database="cloudfront-log.db", read_only=False)
conn.execute('''
                INSTALL 'httpfs';
                LOAD 'httpfs';

                INSTALL json;
                LOAD json;

                SET s3_region='ap-northeast-1';
                SET s3_access_key_id='YOUR_S3_ACCES_KEY';
                SET s3_secret_access_key='YOUR_SECRET_ACCESS_KEY';

                SET enable_progress_bar = true;
            ''')

In [4]:
conn.execute('''
   CREATE TABLE logs (
             date STRING,
             time STRING,
             x_edge_location STRING,
             sc_bytes STRING,
             c_ip STRING,
             cs_method STRING,
             cs_host STRING,
             cs_uri_stem STRING,
             sc_status STRING,
             cs_Referer  STRING,
             cs_User_Agent STRING,
             cs_uri_query STRING,
             cs_Cookie STRING,
             x_edge_result_type STRING,
             x_edge_request_id STRING,
             x_host_header STRING,
             cs_protocol STRING,
             cs_bytes STRING,
             time_taken STRING,
             x_forwarded_for STRING,
             ssl_protocol STRING,
             ssl_cipher STRING,
             x_edge_response_result_type STRING,
             cs_protocol_version STRING,
             fle_status STRING,
             fle_encrypted_fields STRING,
             c_port STRING,
             time_to_first_byte STRING,
             x_edge_detailed_result_type STRING,
             sc_content_type STRING,
             sc_content_len STRING,
             sc_range_start STRING,
             sc_range_end STRING)
     ''')

In [7]:
# COPY logs FROM '{single_file}' (FORMAT csv, DELIMITER '\t', SKIP 2, AUTO_DETECT true, IGNORE_ERRORS true);

In [8]:
# fetch some data
df = conn.execute(f"SELECT * FROM logs limit 10").fetchdf()
print(df.head(10))

Empty DataFrame
Columns: [date, time, x_edge_location, sc_bytes, c_ip, cs_method, cs_host, cs_uri_stem, sc_status, cs_Referer, cs_User_Agent, cs_uri_query, cs_Cookie, x_edge_result_type, x_edge_request_id, x_host_header, cs_protocol, cs_bytes, time_taken, x_forwarded_for, ssl_protocol, ssl_cipher, x_edge_response_result_type, cs_protocol_version, fle_status, fle_encrypted_fields, c_port, time_to_first_byte, x_edge_detailed_result_type, sc_content_type, sc_content_len, sc_range_start, sc_range_end]
Index: []

[0 rows x 33 columns]


In [9]:
sql = f"SELECT \
            x_edge_location, \
            cs_uri_stem, \
            count(*) as cnt \
        FROM logs \
        WHERE x_edge_location ='DUB56-P1' \
        GROUP BY 1, 2 \
        ORDER BY cnt DESC"

df = conn.execute(sql).fetchdf()